# *Conexión al Drive*

In [13]:
# prompt: crear conexion a drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
import sys
path='/content/drive/MyDrive/cod/marketing/MarketingprojectG8/data'
os.chdir(path)###volver la carpeta del repositorio directorio de trabajo
sys.path.append(path)### agregarla al path, para poder leer los archivos de funciones propios como paquetes

# *Librerias*

In [15]:
###paquete para crear y trabajar con bases de datos.
import sqlite3 as sql
import pandas as pd
import numpy as np
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
#import a_funciones as fn
from mlxtend.preprocessing import TransactionEncoder

# *Conexión a SQL*

In [16]:
# Conectarse a la base de datos
con=sql.connect('db_movies')

In [17]:
# Crear un cursor
cur=con.cursor()### otro tipo de conexión para ejecutar consultas en la bd sin traer ni llevar información

In [23]:
# Obtener una lista de tablas en la base de datos
cur.execute("""SELECT name FROM sqlite_master WHERE type= 'table'; """)
tables = cur.fetchall()
print("Tablas disponibles:", tables)

Tablas disponibles: [('ratings',), ('movies',)]


### *Cargar las tablas en DataFrames*

In [27]:
# Cargar tabla movies
cur.execute('SELECT * FROM movies')
movies_data = cur.fetchall()
movies_columns = [description[0] for description in cur.description]  # Obtener nombres de columnas
movies_df = pd.DataFrame(movies_data, columns=movies_columns)


In [28]:
# Mostrar las primeras filas de la tabla
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
# Cargar tabla ratings
cur.execute('SELECT * FROM ratings')
ratings_data = cur.fetchall()
ratings_columns = [description[0] for description in cur.description]  # Obtener nombres de columnas
ratings_df = pd.DataFrame(ratings_data, columns=ratings_columns)

In [30]:
# Mostrar las primeras filas de la tabla
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# *Cerra la conexion*

In [ ]:
# Cerrar la conexión
#cur.close()
#con.close()

# *Análisis exploratorio de los datos*